# 📊 Model Results Deep Analysis

Analisi approfondita dei risultati sfruttando **gli artefatti già salvati in `models/`** (metriche, segmentazioni, prediction intervals, worst predictions).

**Prerequisiti**:
- Almeno una run di training/evaluation (`python main.py --config config/config.yaml --steps train evaluate`) *oppure* la disponibilità dei file già inclusi nel repository
- Nessun bisogno di rigenerare feature o modelli: il notebook legge direttamente i JSON/CSV salvati

**Analisi**:
1. **Leaderboard**: confronto rapido fra modelli, baseline ed ensemble
2. **Metriche Train/Test**: approfondimento del modello selezionato
3. **Validation Curve**: lettura di `validation_results.csv`
4. **Segmentazioni degli errori**: per fascia di prezzo, Zona OMI, Tipologia Edilizia, Categoria Catastale
5. **Worst Predictions**: casi peggiori con residui e % errore
6. **Prediction Intervals**: copertura e ampiezza degli intervalli 80% / 90%

**Output**: report e grafici in `model_analysis_outputs/`

## 🔧 Setup

In [5]:
# Imports
import json
import sys
from pathlib import Path

sys.path.insert(0, str(Path.cwd().parent / "src"))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from IPython.display import display

from utils.config import load_config

warnings.filterwarnings("ignore")
pd.options.display.float_format = "{:.2f}".format

# Plot settings
plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("husl")
%matplotlib inline

print("✅ Setup completato")

✅ Setup completato


In [ ]:
# Configurazione
CONFIG_PATH = Path("../config/config.yaml")
MODELS_DIR = Path("../models")
OUTPUT_DIR = Path("model_analysis_outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

SUMMARY_CANDIDATES = [
    MODELS_DIR / "summary.json",
    MODELS_DIR / "results_summary.json"
]
EVALUATION_SUMMARY_PATH = MODELS_DIR / "evaluation_summary.json"
VALIDATION_RESULTS_PATH = MODELS_DIR / "validation_results.csv"


def first_existing(paths):
    for path in paths:
        if path and path.exists():
            return path
    return None


def load_json_safe(path, default=None):
    if path and path.exists():
        with open(path, "r") as fp:
            return json.load(fp)
    return default if default is not None else {}


def load_csv_optional(path, **kwargs):
    if path.exists():
        return pd.read_csv(path, **kwargs)
    print(f"⚠️ File non trovato: {path}")
    return pd.DataFrame()


def save_plot(name, dpi=120):
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / f"{name}.png", dpi=dpi, bbox_inches="tight")
    print(f"💾 Salvato: {name}.png")


SUMMARY_PATH = first_existing(SUMMARY_CANDIDATES)

print(f"📂 Output directory: {OUTPUT_DIR}")
print(f"📂 Models directory: {MODELS_DIR}")
print(f"📄 Summary file: {SUMMARY_PATH if SUMMARY_PATH else 'non trovato'}")

📂 Output directory: model_analysis_outputs
📂 Preprocessed data: ..\data\preprocessed
📂 Models directory: ..\models


## 📦 1. Caricamento artefatti e selezione del modello

In [7]:
# Caricamento config e artefatti
config = load_config(CONFIG_PATH)
summary = load_json_safe(SUMMARY_PATH, default={})
evaluation_summary = load_json_safe(EVALUATION_SUMMARY_PATH, default={})
validation_df = load_csv_optional(VALIDATION_RESULTS_PATH)


def build_leaderboard(summary_dict):
    rows = []
    sections = [
        ("models", "Optimized"),
        ("ensembles", "Ensemble"),
        ("baselines", "Baseline"),
    ]
    for section, label in sections:
        for name, payload in summary_dict.get(section, {}).items():
            metrics = payload.get("metrics_test", {})
            rows.append(
                {
                    "Model": name,
                    "Source": label,
                    "R2": metrics.get("r2"),
                    "RMSE": metrics.get("rmse"),
                    "MAE": metrics.get("mae"),
                }
            )
    df = pd.DataFrame(rows)
    if df.empty:
        return df
    return df.dropna(subset=["R2"]).sort_values("R2", ascending=False)


def normalize_model_name(label):
    if not label:
        return None
    return label.split("_")[-1].lower()


def pick_best_model(leaderboard, candidates):
    if not leaderboard.empty:
        best_candidate = leaderboard.iloc[0]["Model"]
        if best_candidate in candidates:
            return best_candidate
        normalized = normalize_model_name(best_candidate)
        if normalized in candidates:
            return normalized
    return None


def pick_from_validation(df, candidates):
    if df.empty:
        return None
    best_row = df.sort_values("Test_R2", ascending=False).iloc[0]
    candidate = normalize_model_name(best_row["Model"])
    return candidate if candidate in candidates else None


leaderboard_df = build_leaderboard(summary)
available_model_dirs = sorted([p.name for p in MODELS_DIR.iterdir() if p.is_dir()])
print(f"📁 Modelli disponibili: {', '.join(available_model_dirs) if available_model_dirs else 'nessuno'}")

selected_model_key = pick_best_model(leaderboard_df, available_model_dirs)
if not selected_model_key:
    selected_model_key = pick_from_validation(validation_df, available_model_dirs)
if not selected_model_key and available_model_dirs:
    selected_model_key = available_model_dirs[0]

if not selected_model_key:
    raise FileNotFoundError("Nessuna directory modello disponibile sotto models/")

SELECTED_MODEL_KEY = selected_model_key
MODEL_ARTIFACT_DIR = MODELS_DIR / SELECTED_MODEL_KEY
print(f"🏆 Modello analizzato: {SELECTED_MODEL_KEY}")

model_metrics = load_json_safe(MODEL_ARTIFACT_DIR / "metrics.json", default={})
price_band_df = load_csv_optional(MODEL_ARTIFACT_DIR / "group_metrics_price_band.csv")
zona_metrics_df = load_csv_optional(MODEL_ARTIFACT_DIR / "group_metrics_AI_ZonaOmi.csv")
tipologia_metrics_df = load_csv_optional(MODEL_ARTIFACT_DIR / "group_metrics_AI_IdTipologiaEdilizia.csv")
categoria_metrics_df = load_csv_optional(MODEL_ARTIFACT_DIR / "group_metrics_AI_IdCategoriaCatastale.csv")
worst_predictions_df = load_csv_optional(MODEL_ARTIFACT_DIR / f"{SELECTED_MODEL_KEY}_worst_predictions.csv")
prediction_intervals = load_json_safe(
    MODEL_ARTIFACT_DIR / f"{SELECTED_MODEL_KEY}_prediction_intervals.json", default={}
)

print("✅ Config e artefatti caricati\n")
print(f"📊 Leaderboard entries: {len(leaderboard_df)}")
print(f"📈 Validation rows: {len(validation_df)}")
print(f"📦 Metriche disponibili: {'sì' if model_metrics else 'no'}")

✅ Config e results caricati

📊 Best Model: N/A
📊 Best Params: {}


In [8]:
# Leaderboard overview
if leaderboard_df.empty:
    print("⚠️ Nessuna metrica trovata in summary.json/results_summary.json")
else:
    leaderboard_display = leaderboard_df.copy()
    leaderboard_display[["R2", "RMSE", "MAE"]] = leaderboard_display[["R2", "RMSE", "MAE"]].round(3)
    print("🏁 Leaderboard globale (Top 10 per R²)")
    display(leaderboard_display.head(10))
    leaderboard_display.to_csv(OUTPUT_DIR / "01_leaderboard.csv", index=False)
    print("💾 Salvato: 01_leaderboard.csv")
    
    top_plot = leaderboard_display.head(8)
    fig, ax = plt.subplots(figsize=(10, 5))
    sns.barplot(data=top_plot, x="Model", y="R2", hue="Source", ax=ax)
    ax.set_ylim(0, 1)
    ax.set_ylabel("R² (test)")
    ax.set_xlabel("")
    ax.set_title("R² test per modello", fontsize=14, fontweight="bold")
    ax.grid(True, axis="y", alpha=0.3)
    plt.xticks(rotation=30, ha="right")
    save_plot("02_leaderboard_r2")
    plt.show()

✅ Preprocessed data caricati

📊 Shapes:
   Train: X=(3897, 370), y=(3897,)
   Val:   X=(1136, 370), y=(1136,)
   Test:  X=(569, 370), y=(569,)


In [9]:
# Metriche train/test del modello selezionato
if not model_metrics:
    print("⚠️ metrics.json non trovato per il modello selezionato")
else:
    metrics_rows = []
    for split_name, split_key in [("Train", "metrics_train"), ("Test", "metrics_test")]:
        split_metrics = model_metrics.get(split_key, {})
        if not split_metrics:
            continue
        metrics_rows.append(
            {
                "Split": split_name,
                "R2": split_metrics.get("r2"),
                "RMSE": split_metrics.get("rmse"),
                "MAE": split_metrics.get("mae"),
                "MAPE": split_metrics.get("mape"),
                "MedAE": split_metrics.get("medae"),
            }
        )
    metrics_df = pd.DataFrame(metrics_rows)
    if metrics_df.empty:
        print("⚠️ Nessuna metrica disponibile nel file metrics.json")
    else:
        metrics_df_round = metrics_df.copy()
        metrics_df_round[["R2", "RMSE", "MAE", "MAPE", "MedAE"]] = metrics_df_round[
            ["R2", "RMSE", "MAE", "MAPE", "MedAE"]
        ].round(3)
        display(metrics_df_round)
        metrics_df_round.to_csv(OUTPUT_DIR / f"03_{SELECTED_MODEL_KEY}_metrics.csv", index=False)
        print(f"💾 Salvato: 03_{SELECTED_MODEL_KEY}_metrics.csv")
        
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
        sns.barplot(data=metrics_df, x="Split", y="RMSE", ax=axes[0], palette="viridis")
        axes[0].set_title("RMSE (test vs train)")
        axes[0].grid(True, axis="y", alpha=0.3)
        sns.barplot(data=metrics_df, x="Split", y="MAE", ax=axes[1], palette="magma")
        axes[1].set_title("MAE (test vs train)")
        axes[1].grid(True, axis="y", alpha=0.3)
        save_plot(f"04_{SELECTED_MODEL_KEY}_metrics")
        plt.show()
        
        best_params = model_metrics.get("best_params", {})
        if best_params:
            print("⚙️ Best params:")
            for k, v in best_params.items():
                print(f"   - {k}: {v}")

⚠️  Best model non trovato, provo con modelli individuali...
❌ ERRORE: Nessun modello trovato!


FileNotFoundError: Nessun modello trovato

## 📊 2. Validation results & stability

In [ ]:
# Validation results overview
if validation_df.empty:
    print("⚠️ validation_results.csv non trovato")
else:
    val_display = validation_df.copy()
    val_display[["Test_RMSE", "Test_R2"]] = val_display[["Test_RMSE", "Test_R2"]].astype(float).round(4)
    display(val_display)
    val_display.to_csv(OUTPUT_DIR / "05_validation_results.csv", index=False)
    print("💾 Salvato: 05_validation_results.csv")
    
    fig, ax = plt.subplots(figsize=(8, 5))
    sns.scatterplot(
        data=validation_df,
        x="Test_RMSE",
        y="Test_R2",
        hue="Category",
        s=120,
        ax=ax
    )
    for _, row in validation_df.iterrows():
        ax.text(row["Test_RMSE"] + 0.05, row["Test_R2"] + 0.002, row["Model"], fontsize=8)
    ax.set_xlabel("Test RMSE")
    ax.set_ylabel("Test R²")
    ax.set_title("Validation results (RMSE vs R²)")
    ax.grid(True, alpha=0.3)
    save_plot("06_validation_scatter")
    plt.show()

In [ ]:
# Errori per fascia di prezzo (Price band)
if price_band_df.empty:
    print("⚠️ group_metrics_price_band.csv non trovato")
else:
    price_band_display = price_band_df.copy()
    metric_cols = ["r2", "rmse", "mae", "mape", "medae"]
    price_band_display[metric_cols] = price_band_display[metric_cols].astype(float).round(3)
    price_band_display.rename(columns={"group": "Price_Range", "count": "Count", "mape": "MAPE"}, inplace=True)
    print("📊 Errori per fascia di prezzo (test set)")
    display(price_band_display)
    price_band_display.to_csv(OUTPUT_DIR / f"07_{SELECTED_MODEL_KEY}_price_band.csv", index=False)
    print(f"💾 Salvato: 07_{SELECTED_MODEL_KEY}_price_band.csv")

In [ ]:
# Visualizzazione errori per fascia di prezzo
if price_band_df.empty:
    print("⚠️ Nessun dato per la fascia di prezzo")
else:
    fig, axes = plt.subplots(1, 2, figsize=(16, 5))
    order = price_band_df.sort_values("rmse", ascending=False)["group"].tolist()
    sns.barplot(data=price_band_df, x="group", y="rmse", ax=axes[0], order=order, color="steelblue")
    axes[0].set_ylabel("RMSE (€)")
    axes[0].set_xlabel("Fascia di prezzo")
    axes[0].set_title("RMSE per fascia di prezzo")
    axes[0].tick_params(axis="x", rotation=60)
    axes[0].grid(True, axis="y", alpha=0.3)
    
    sns.barplot(data=price_band_df, x="group", y="mape", ax=axes[1], order=order, color="darkorange")
    axes[1].set_ylabel("MAPE")
    axes[1].set_xlabel("Fascia di prezzo")
    axes[1].set_title("MAPE per fascia di prezzo")
    axes[1].tick_params(axis="x", rotation=60)
    axes[1].grid(True, axis="y", alpha=0.3)
    
    plt.suptitle("Distribuzione degli errori per fascia di prezzo", fontsize=16, fontweight="bold")
    save_plot(f"08_{SELECTED_MODEL_KEY}_price_band")
    plt.show()

## 📊 3. Errori per Zona OMI

In [ ]:
# Metriche per Zona OMI
if zona_metrics_df.empty:
    print("⚠️ group_metrics_AI_ZonaOmi.csv non trovato")
else:
    zona_display = zona_metrics_df.copy()
    zona_display.rename(columns={"group": "ZonaOMI", "count": "Count", "mape": "MAPE"}, inplace=True)
    zona_display[["r2", "rmse", "mae", "MAPE"]] = zona_display[["r2", "rmse", "mae", "MAPE"]].astype(float).round(3)
    zona_display = zona_display.sort_values("rmse", ascending=False)
    display(zona_display)
    zona_display.to_csv(OUTPUT_DIR / f"09_{SELECTED_MODEL_KEY}_zona_omi.csv", index=False)
    print(f"💾 Salvato: 09_{SELECTED_MODEL_KEY}_zona_omi.csv")

In [ ]:
# Visualizzazione Zona OMI
if zona_metrics_df.empty:
    print("⚠️ Nessun dato Zona OMI")
else:
    top_zona = zona_metrics_df.sort_values("rmse", ascending=False).head(10)
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.barplot(data=top_zona, x="rmse", y="group", palette="crest", ax=ax)
    ax.set_xlabel("RMSE (€)")
    ax.set_ylabel("Zona OMI")
    ax.set_title("Top 10 zone per errore (RMSE)")
    ax.grid(True, axis="x", alpha=0.3)
    save_plot(f"10_{SELECTED_MODEL_KEY}_zona_omi")
    plt.show()

In [ ]:
# Metriche per Tipologia Edilizia
if tipologia_metrics_df.empty:
    print("⚠️ group_metrics_AI_IdTipologiaEdilizia.csv non trovato")
else:
    tipologia_display = tipologia_metrics_df.copy()
    tipologia_display.rename(columns={"group": "Tipologia", "count": "Count", "mape": "MAPE"}, inplace=True)
    tipologia_display[["r2", "rmse", "mae", "MAPE"]] = tipologia_display[["r2", "rmse", "mae", "MAPE"]].astype(float).round(3)
    tipologia_display = tipologia_display.sort_values("MAPE", ascending=False)
    display(tipologia_display)
    tipologia_display.to_csv(OUTPUT_DIR / f"11_{SELECTED_MODEL_KEY}_tipologia.csv", index=False)
    print(f"💾 Salvato: 11_{SELECTED_MODEL_KEY}_tipologia.csv")

## 📊 4. Tipologia Edilizia (errori)

In [ ]:
# Visualizzazione tipologie
if tipologia_metrics_df.empty:
    print("⚠️ Nessun dato sulla tipologia")
else:
    fig, ax = plt.subplots(figsize=(10, 6))
    top_tipologie = tipologia_metrics_df.sort_values("mape", ascending=False).head(12)
    sns.barplot(data=top_tipologie, x="mape", y="group", palette="flare", ax=ax)
    ax.set_xlabel("MAPE")
    ax.set_ylabel("Tipologia edilizia")
    ax.set_title("Top tipologie per errore percentuale")
    ax.grid(True, axis="x", alpha=0.3)
    save_plot(f"12_{SELECTED_MODEL_KEY}_tipologia")
    plt.show()

## 📊 5. Categoria Catastale

In [ ]:
# Metriche per Categoria Catastale
if categoria_metrics_df.empty:
    print("⚠️ group_metrics_AI_IdCategoriaCatastale.csv non trovato")
else:
    categoria_display = categoria_metrics_df.copy()
    categoria_display.rename(columns={"group": "CategoriaCatastale", "count": "Count", "mape": "MAPE"}, inplace=True)
    categoria_display[["r2", "rmse", "mae", "MAPE"]] = categoria_display[["r2", "rmse", "mae", "MAPE"]].astype(float).round(3)
    categoria_display = categoria_display.sort_values("rmse", ascending=False)
    display(categoria_display)
    categoria_display.to_csv(OUTPUT_DIR / f"13_{SELECTED_MODEL_KEY}_categoria.csv", index=False)
    print(f"💾 Salvato: 13_{SELECTED_MODEL_KEY}_categoria.csv")

In [ ]:
# Visualizzazione categorie catastali
if categoria_metrics_df.empty:
    print("⚠️ Nessun dato categoria catastale")
else:
    fig, ax = plt.subplots(figsize=(10, 6))
    top_categorie = categoria_metrics_df.sort_values("rmse", ascending=False).head(10)
    sns.barplot(data=top_categorie, x="rmse", y="group", palette="rocket", ax=ax)
    ax.set_xlabel("RMSE (€)")
    ax.set_ylabel("Categoria catastale")
    ax.set_title("Categorie con errore più elevato")
    ax.grid(True, axis="x", alpha=0.3)
    save_plot(f"14_{SELECTED_MODEL_KEY}_categoria")
    plt.show()

## 📊 6. Worst Predictions Analysis

In [ ]:
# Worst predictions (già salvate in fase di training)
if worst_predictions_df.empty:
    print("⚠️ Nessun file *_worst_predictions.csv trovato per il modello selezionato")
else:
    worst_display = worst_predictions_df.copy()
    worst_display.rename(
        columns={"true": "Actual", "predicted": "Predicted", "residual": "Residual", "abs_residual": "Abs_Error", "pct_error": "Pct_Error"},
        inplace=True,
    )
    worst_display[["Actual", "Predicted", "Residual", "Abs_Error"]] = worst_display[
        ["Actual", "Predicted", "Residual", "Abs_Error"]
    ].astype(float).round(2)
    worst_display["Pct_Error"] = worst_display["Pct_Error"].astype(float).round(2)
    display(worst_display.head(20))
    worst_display.to_csv(OUTPUT_DIR / f"15_{SELECTED_MODEL_KEY}_worst_predictions.csv", index=False)
    print(f"💾 Salvato: 15_{SELECTED_MODEL_KEY}_worst_predictions.csv")

## 📊 7. Prediction Intervals

In [ ]:
# Prediction intervals (coverage vs target)
if not prediction_intervals:
    print("⚠️ File *_prediction_intervals.json non trovato o vuoto")
else:
    interval_rows = []
    for level, stats_dict in prediction_intervals.items():
        interval_rows.append(
            {
                "Interval": level,
                "Target": stats_dict.get("target_coverage"),
                "Coverage": stats_dict.get("coverage"),
                "AvgWidth€": stats_dict.get("average_width"),
                "AvgWidth%": stats_dict.get("average_width_pct"),
            }
        )
    interval_df = pd.DataFrame(interval_rows).sort_values("Target")
    interval_df[["Target", "Coverage"]] = interval_df[["Target", "Coverage"]].round(3)
    interval_df[["AvgWidth€", "AvgWidth%"]] = interval_df[["AvgWidth€", "AvgWidth%"]].round(2)
    display(interval_df)
    interval_df.to_csv(OUTPUT_DIR / f"16_{SELECTED_MODEL_KEY}_prediction_intervals.csv", index=False)
    print(f"💾 Salvato: 16_{SELECTED_MODEL_KEY}_prediction_intervals.csv")
    
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.plot(interval_df["Target"], interval_df["Target"], "k--", label="Target")
    ax.plot(interval_df["Target"], interval_df["Coverage"], marker="o", label="Coverage")
    ax.set_xlabel("Target coverage")
    ax.set_ylabel("Observed coverage")
    ax.set_title("Prediction interval coverage")
    ax.set_ylim(0, 1)
    ax.grid(True, alpha=0.3)
    ax.legend()
    save_plot(f"17_{SELECTED_MODEL_KEY}_prediction_intervals")
    plt.show()

## 📋 8. Summary Report

In [ ]:
# Report finale (aggregato dagli artefatti)

def top_records(df, sort_col, n=3):
    if df is None or df.empty:
        return []
    return df.sort_values(sort_col, ascending=False).head(n).to_dict("records")


summary_report = {
    "selected_model": SELECTED_MODEL_KEY,
    "available_models": available_model_dirs,
    "leaderboard_top3": leaderboard_df.head(3).to_dict("records") if not leaderboard_df.empty else [],
    "metrics": {
        "train": model_metrics.get("metrics_train", {}),
        "test": model_metrics.get("metrics_test", {}),
        "best_params": model_metrics.get("best_params", {}),
    },
    "validation_rows": int(len(validation_df)),
    "segments": {
        "price_band_high_mape": top_records(price_band_df, "mape"),
        "zona_high_rmse": top_records(zona_metrics_df, "rmse"),
        "tipologia_high_mape": top_records(tipologia_metrics_df, "mape"),
        "categoria_high_rmse": top_records(categoria_metrics_df, "rmse"),
    },
    "prediction_intervals": prediction_intervals,
    "worst_prediction_sample": worst_predictions_df.head(1).to_dict("records")[0]
    if not worst_predictions_df.empty
    else None,
}

summary_path = OUTPUT_DIR / f"18_{SELECTED_MODEL_KEY}_summary_report.json"
with open(summary_path, "w") as f:
    json.dump(summary_report, f, indent=2)

print("\n" + "=" * 80)
print("📋 FINAL REPORT")
print("=" * 80)
print(json.dumps(summary_report, indent=2))
print(f"\n💾 Salvato: {summary_path.name}")

## ✅ Conclusioni

### File generati

- `01_leaderboard.csv` / `02_leaderboard_r2.png`: confronto rapido di tutti i modelli
- `03_<model>_metrics.csv` / `04_<model>_metrics.png`: metriche train/test del modello scelto
- `05_validation_results.csv` / `06_validation_scatter.png`: performance sulle run di validazione
- `07-14_*` CSV/PNG: segmentazioni per fascia prezzo, Zona OMI, Tipologia, Categoria catastale
- `15_<model>_worst_predictions.csv`: elenco dei casi più critici con residuo e % errore
- `16_<model>_prediction_intervals.csv` / `17_<model>_prediction_intervals.png`: copertura intervalli
- `18_<model>_summary_report.json`: riepilogo finale (metriche, segmentazioni, intervalli)

### Key insights

- **Leaderboard**: il modello selezionato è quello con R² più alto nelle metriche salvate; le alternative sono comunque disponibili per confronto rapido
- **Segmentazioni**: le fasce di prezzo e le categorie catastali con RMSE/MAPE più elevati emergono subito dai CSV dedicati
- **Prediction intervals**: il grafico mette in evidenza l’eventuale under/over-coverage rispetto al target (80%/90%)
- **Worst predictions**: disponibile un campione pronto per debugging qualitativo o per creare casi di studio

### Next steps

1. Concentrarsi sulle fasce/categorie con MAPE più alto per capire possibili feature aggiuntive
2. Se la copertura degli intervalli è bassa, aumentare `n_bootstraps` o estendere il livello di confidenza
3. Usare il file dei worst predictions per effettuare controlli di qualità sui dati sorgente o sulle etichette
4. Condividere i grafici/CSV generati con il team business per raccogliere feedback mirati